In [ ]:
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 3.6 MB/s eta 0:00:00
  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.7/165.7 kB 9.2 MB/s eta 0:00:00
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 53.7 MB/s eta 0:00:00
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.0
    Uninstalling tokenizers-0.20.0:
      Successfully uninstalled tokenizers-0.20.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Un

Found existing installation: unsloth 2024.9.post4
Uninstalling unsloth-2024.9.post4:
  Successfully uninstalled unsloth-2024.9.post4
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-nmiez26h/unsloth_517085a48efd45c48f2d6429dae5b35f
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-nmiez26h/unsloth_517085a48efd45c48f2d6429dae5b35f
ERROR: Operation cancelled by user


In [ ]:
!pip install -q -U transformers
!pip install -q -U bitsandbytes
!pip install -q -U peft
!pip install -q -U trl
!pip install -q -U accelerate

In [ ]:
# Huggingface 로그인
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoTokenizer

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("unsloth/gemma-2-2b")
tokenizer.model_max_length = 512

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from transformers import BitsAndBytesConfig

# BitsAndBytesConfig 객체 생성 (8비트 양자화 설정)
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # 8비트 양자화 설정
    llm_int8_threshold=6.0  # 양자화된 레이어의 활성화 임계값 (기본값)
)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

# 감정 분류를 위한 모델 설정
emotion_classification_model = AutoModelForSequenceClassification.from_pretrained(
    "unsloth/gemma-2-2b",
    num_labels=6,  # 감정 레이블 수 (0~5)
    device_map="auto",  # GPU 자동 할당
    quantization_config=bnb_config  # 8비트 양자화 적용
)

config.json:   0%|          | 0.00/891 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/5.23G [00:00<?, ?B/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at unsloth/gemma-2-2b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import datasets

train_dataset = datasets.load_from_disk('/content/drive/MyDrive/mlb_gemma/datasets/train')
valid_dataset = datasets.load_from_disk('/content/drive/MyDrive/mlb_gemma/datasets/valid')
test_dataset = datasets.load_from_disk('/content/drive/MyDrive/mlb_gemma/datasets/test')

# 출력 확인
print(f"Train set: {len(train_dataset)} samples")
print(f"Validation set: {len(valid_dataset)} samples")
print(f"Test set: {len(test_dataset)} samples")

Train set: 300102 samples
Validation set: 33345 samples
Test set: 83362 samples


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
from transformers import TrainingArguments

# LoRA 설정
lora_config = LoraConfig(
    r=16,                        # 랭크 설정
    lora_alpha=32,               # 학습률 스케일링
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],  # 어댑터를 추가할 모듈
    lora_dropout=0.05,           # 드롭아웃 비율
    bias="none",                 # 바이어스 설정
    task_type="CAUSAL_LM"        # 언어 모델링 태스크
)

# LoRA 어댑터를 모델에 추가
emotion_classification_model = get_peft_model(emotion_classification_model, lora_config)

In [ ]:
# 학습 파라미터 설정
training_args = TrainingArguments(
    output_dir = "/content/drive/MyDrive/mlb_gemma/emotion_model/",
    per_device_train_batch_size=2,    # 각 장치에서 학습할 배치 크기
    gradient_accumulation_steps=4,     # 기울기 누적 단계 수
    fp16 = True, # FP16(반정밀도) 사용 여부 (bfloat16을 사용할 수 없을 경우 FP16 사용)
    #bf16 = True, # bfloat16 사용 여부
    warmup_steps = 5,               # 학습 초반 워밍업 스텝 수 (학습 초기에 낮은 학습률 적용)
    learning_rate=2e-5,                # 학습률
    weight_decay=0.01,                 # 가중치 감쇠
    optim="adamw_8bit",                # 8비트 AdamW 옵티마이저 사용
    lr_scheduler_type="linear",        # 선형 학습률 스케줄러 사용
    #num_train_epochs=1,                # 학습할 에포크 수
    max_steps = 100,                     # 전체 학습 스텝 수 (지정된 스텝만큼 학습 진행)
    logging_steps=1,                   # 로그 출력 간격
    seed=3407                          # 시드 값 (재현성을 위해 설정)
)

# Trainer 설정
trainer = Trainer(
    model=emotion_classification_model,
    args=training_args,
    train_dataset=train_dataset,  # 감정 분류를 위한 학습 데이터셋
    eval_dataset=valid_dataset,   # 검증 데이터셋
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# 감정 분류 학습 시작
trainer.train()

Step,Training Loss
1,2.991800
2,2.158800
3,1.557300
4,2.751800
5,2.158700
6,2.038400
7,2.060300
8,1.475900
9,1.188200
10,2.309500


TrainOutput(global_step=100, training_loss=2.352126429080963, metrics={'train_runtime': 451.0156, 'train_samples_per_second': 1.774, 'train_steps_per_second': 0.222, 'total_flos': 5026525426483200.0, 'train_loss': 2.352126429080963, 'epoch': 0.0026657603081618915})

In [ ]:
# 학습 완료 후 test 데이터셋 평가
trainer.evaluate(test_dataset)